This file shows how I produced the prediction results that were submitted on Kaggle. 

After comparing the prediction performance of different models, I decided to use 
random forest and gradient boosting for the final preidctions with the same traning data.
I submited the prediction results on Kaggle and got the following results:

The score(root mean square error) of each model:
- predictions made by the random forest model:1.27898
- predictions made by the gradient boosting model: 0.96080
- predictions made by the winner of this competition 3 years ago: 0.88140




In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, SparsePCA
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


from sklearn.model_selection import GridSearchCV


from sklearn.metrics import mean_squared_error



## DATA PREPROCESSING - test set

In [2]:
#test_clean.csv came from test.csv
test = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/test_v2_withid.csv', low_memory=False).drop(['Unnamed: 0'], axis=1)

In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 27 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   visitNumber                               401589 non-null  int64  
 1   totals.visits                             401589 non-null  int64  
 2   totals.hits                               401589 non-null  int64  
 3   totals.pageviews                          401488 non-null  float64
 4   totals.timeOnSite                         217903 non-null  float64
 5   totals.newVisits                          286065 non-null  float64
 6   totals.transactions                       6305 non-null    float64
 7   totals.totalTransactionRevenue            4594 non-null    float64
 8   totals.bounces                            182678 non-null  float64
 9   channelGrouping                           401589 non-null  object 
 10  socialEngagementType

In [4]:
ID = test['fullVisitorId']

In [5]:
test_x = test.drop(['fullVisitorId'], axis=1)

In [6]:
test_x.shape

(401589, 26)

In [7]:
test_x.head()

,visitNumber,totals.visits,totals.hits,totals.pageviews,totals.timeOnSite,totals.newVisits,totals.transactions,totals.totalTransactionRevenue,totals.bounces,channelGrouping,...,geoNetwork.subContinent,geoNetwork.country,geoNetwork.networkDomain,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.referralPath,trafficSource.isTrueDirect
0,2,1,4,3.0,973.0,NaN,NaN,NaN,NaN,Organic Search,...,Southern Asia,India,unknown.unknown,(not set),google,organic,(not provided),NaN,(not set),True
1,166,1,4,3.0,49.0,NaN,NaN,NaN,NaN,Direct,...,Northern America,United States,(not set),(not set),(direct),(none),(not set),NaN,(not set),True
2,2,1,4,3.0,24.0,NaN,NaN,NaN,NaN,Organic Search,...,Northern America,United States,onlinecomputerworks.com,(not set),google,organic,(not provided),NaN,(not set),True
3,4,1,5,4.0,25.0,NaN,NaN,NaN,NaN,Direct,...,Northern America,United States,(not set),(not set),(direct),(none),(not set),NaN,(not set),True
4,1,1,5,4.0,49.0,1.0,NaN,NaN,NaN,Organic Search,...,Northern America,United States,com,(not set),google,organic,(not provided),NaN,(not set),NaN


In [8]:
#help decide which columns to drop due to missing values
def Null_Count(df):
    df_null = df.isnull().sum().sort_values(ascending = False).rename('Null').reset_index()

    null_count = df_null['Null']
    null_percent = (null_count * 100) / (df.shape[0])

    df_null = pd.concat([df_null['index'],null_count,null_percent], axis=1, keys=['Column','Null_Count','Null_Percent'])

    return df_null[df_null['Null_Count'] != 0]

Null_Count(test_x)

,Column,Null_Count,Null_Percent
0,totals.totalTransactionRevenue,396995,98.856044
1,totals.transactions,395284,98.429987
2,trafficSource.adwordsClickInfo.isVideoAd,390984,97.359240
3,trafficSource.isTrueDirect,253180,63.044556
4,totals.bounces,218911,54.511204
5,totals.timeOnSite,183686,45.739799
6,totals.newVisits,115524,28.766724
7,trafficSource.keyword,40226,10.016709
8,totals.pageviews,101,0.025150


In [9]:
def fill_na(df):   
    df["totals.totalTransactionRevenue"].fillna(0.0, inplace=True)
    df["totals.transactions"].fillna(0.0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True) 
    df['totals.timeOnSite'].fillna(0, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.pageviews'].fillna(1, inplace=True)

    
    # Changing datatypes from object to desired ones
    df['totals.pageviews'] = df['totals.pageviews'].astype(int)
    df['totals.newVisits'] = df['totals.newVisits'].astype(int)
    df['totals.bounces'] = df['totals.bounces'].astype(int)
    df["totals.totalTransactionRevenue"] = df["totals.totalTransactionRevenue"].astype(float)
    df["totals.transactions"] = df["totals.totalTransactionRevenue"].astype(float)

    
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True)
    df['trafficSource.keyword'].fillna('(not provided)', inplace=True) 

    return df

test_x = fill_na(test_x)

In [10]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 26 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   visitNumber                               401589 non-null  int64  
 1   totals.visits                             401589 non-null  int64  
 2   totals.hits                               401589 non-null  int64  
 3   totals.pageviews                          401589 non-null  int64  
 4   totals.timeOnSite                         401589 non-null  float64
 5   totals.newVisits                          401589 non-null  int64  
 6   totals.transactions                       401589 non-null  float64
 7   totals.totalTransactionRevenue            401589 non-null  float64
 8   totals.bounces                            401589 non-null  int64  
 9   channelGrouping                           401589 non-null  object 
 10  socialEngagementType

In [11]:
num_name = ['visitNumber', 'totals.visits', 'totals.hits', \
            'totals.pageviews', 'totals.timeOnSite', 'totals.newVisits', \
            'totals.transactions', 'totals.totalTransactionRevenue', 'totals.bounces']
num_c = test_x[num_name]
cat_c = test_x.drop(num_name, axis=1)

In [12]:
for i in range(len(cat_c.columns)):
    cat_c.iloc[:,i] = LabelEncoder().fit_transform(cat_c.iloc[:,i])

In [13]:
X_total = pd.concat([num_c, cat_c], axis=1)

In [14]:
real_x = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/X_v2.csv', \
                low_memory=False).drop(['visitHour', 'Unnamed: 0'], axis=1)

In [15]:
X_total.columns.to_list() == real_x.columns.to_list()

True

In [16]:
X_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 26 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   visitNumber                               401589 non-null  int64  
 1   totals.visits                             401589 non-null  int64  
 2   totals.hits                               401589 non-null  int64  
 3   totals.pageviews                          401589 non-null  int64  
 4   totals.timeOnSite                         401589 non-null  float64
 5   totals.newVisits                          401589 non-null  int64  
 6   totals.transactions                       401589 non-null  float64
 7   totals.totalTransactionRevenue            401589 non-null  float64
 8   totals.bounces                            401589 non-null  int64  
 9   channelGrouping                           401589 non-null  int64  
 10  socialEngagementType

In [17]:
X_unique = X_total
X_unique['fullVisitorId'] = ID

In [18]:
X_unique.drop_duplicates(subset ='fullVisitorId',
                     keep = 'first', inplace = True)
  

In [19]:
X_noid = X_unique.drop('fullVisitorId', axis = 1)

In [20]:
scaler = MinMaxScaler()
X_s = scaler.fit_transform(X_noid)

pca_n = PCA(n_components = 12)
X_r = pca_n.fit_transform(X_s)

In [21]:
np.sum(pca_n.explained_variance_ratio_)> 0.95

True

## DATA PREPROCESSING - train set

In [22]:
X = real_x
y = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/y_v2.csv').drop('Unnamed: 0', axis=1)


In [23]:
y_log = np.log(y + 1)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.3, random_state=42)

In [24]:
#remeber to reshape y to 1d array, otherwise warning message would be raised
y_train = y_train.to_numpy().reshape(len(y_train),)

In [25]:
#standardize X and do PCA
scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)

pca_train = PCA(n_components = 12)
X_train_r = pca_train.fit_transform(X_train_s)

## MODEL FITTING

In [26]:
#The gradient boosting model
m_gb = GradientBoostingRegressor(n_estimators=70, max_depth=20, min_samples_leaf=20,\
                                  loss='ls', learning_rate=0.05, criterion='mse',\
                                  max_features='sqrt', random_state=2)
m_gb.fit(X_train_r, y_train)


GradientBoostingRegressor(criterion='mse', learning_rate=0.05, max_depth=20,
                          max_features='sqrt', min_samples_leaf=20,
                          n_estimators=70, random_state=2)

In [27]:
#The random forest model
rf_b = RandomForestRegressor(max_depth=35, max_features='sqrt', random_state=2, bootstrap=False)
rf_b.fit(X_train_r, y_train)

RandomForestRegressor(bootstrap=False, max_depth=35, max_features='sqrt',
                      random_state=2)

## PREDICTION

In [28]:
pred_1 = m_gb.predict(X_r)
pred_2 = rf_b.predict(X_r)


## SUBMISSION

In [29]:
#check the format of the sample submission
sample = pd.read_csv('/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/sample_submission_v2.csv',\
                    low_memory=False).drop('PredictedLogRevenue', axis=1)

In [30]:
sample['fullVisitorId']

0         0000018966949534117
1         0000039738481224681
2         0000073585230191399
3         0000087588448856385
4         0000149787903119437
                 ...         
296525    9999862054614696520
296526    9999898168621645223
296527     999990167740728398
296528    9999915620249883537
296529    9999947552481876143
Name: fullVisitorId, Length: 296530, dtype: object

### export gradient boosting results

In [31]:
submission1 = pd.DataFrame()
submission1['fullVisitorId'] = X_unique['fullVisitorId']
submission1['PredictedLogRevenue'] = pred_1

In [32]:
submission1['new_id'] = sample['fullVisitorId']

In [33]:
submission1.head()

In [34]:
submission1.sort_values(by=['fullVisitorId'], inplace=True)

In [35]:
submission2 = submission1.drop('new_id', axis=1)

In [36]:
submission2.head()

In [40]:
#export results
#submission2.to_csv(r'/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/Submission2.csv',\
                 #index=False)

### export random forest results

In [41]:
submission3 = pd.DataFrame()
submission3['fullVisitorId'] = X_unique['fullVisitorId']
submission3['PredictedLogRevenue'] = pred_2

#submission3['new_id'] = sample['fullVisitorId']

In [42]:
#submission3.head()

In [43]:
#submission4 = submission3.drop('new_id', axis=1)
#submission4.to_csv(r'/Users/mercuryliu/Documents/Kaggle/ga-customer-revenue-prediction/Submission3.csv',\
                 #index=False)